# 训练深度模型中的优化

我们下面主要关注这一类特定的优化问题：寻找神经网络上的一组参数 $\mathbf{\theta}$，使得代价函数 $J(\mathbf{\theta})$ 显著小。

此处的代价函数一般都包括了整个训练集上的性能评估和正则化项。

## deep models与传统的优化区别

在大多数机器学习问题中，我们最终关注的是性能度量 P，比如准确率或者 MSE 等度量指标，但这些指标很多时候是不可解的，所以我们只是间接优化 P。我们希望通过降低代价函数 $J(\mathbf{\theta})$ 来提高性能指标 P。

这一点和纯优化不同，纯优化的最小化目标就是 J 本身。

通常代价函数可以写成训练集上的平均，比如：

$$J(\mathbf{\theta}) = E_{\;(\mathbf{x},y)\sim \hat{p}_{data}} L(f(\mathbf{x},\mathbf{\theta}),y) \tag{CH3-1}$$

其中 L 是样本的损失函数， $f(\mathbf{x},\mathbf{\theta})$ 是模型输入 $\mathbf{x}$ 预测的输出， $\hat{p}_{data}$ 是经验分布(也就是训练数据对应的分布)，y 是监督学习中的目标输出。当然，我们也可以改变其形式，使其适应不同形式的正则化问题或者无监督问题。

上式定义了训练集上的目标函数，但实际问题中，训练集只是数据的一部分，我们需要预测的是训练集外的其他数据，我们记包含了训练集以及之外数据的真实数据生成分布为 $p_{data}$，我们更希望的是最小化这个分布上的期望：

$$J^\ast(\mathbf{\theta}) = E_{\;(\mathbf{x},y)\sim p_{data}} L(f(\mathbf{x},\mathbf{\theta}),y) \tag{CH3-2}$$

### 经验风险最小化

机器学习算法的理想目标是降低 (CH3-2) 所示的期望泛化误差(因为同时包含了训练集以及其之外的所有数据)，这个目标被称为 **风险(risk)**。但可惜的是，大多数时候我们通常不知道数据产生背后的真实分布 $p_{data}$。

所有我们寄希望于**最小化训练集上的期望损失，也就是(CH3-1)所示。**我们也称(CH3-1)式为 **经验风险(empirical risk)**：

$$E_{\;(\mathbf{x},y)\sim \hat{p}_{data}} L(f(\mathbf{x},\mathbf{\theta}),y) = \frac{1}{m}\sum_{i=1}^m L(f(\mathbf{x},\mathbf{\theta}),y)$$

最小化这种平均误差最小化的训练过程被称为**经验风险最小化（empirical risk minimization）**。最小化经验风险和传统的优化方法并无差别。

**经验风险和真实风险的关系：**

    我们并不直接优化风险(因为不知道真实的数据分布，无法优化)，在不同的理论和条件下，经验风险的降低可以使得真实风险的期望也可以相应地下降。



**最小化经验风险**不适合作为机器学习真正优化目标的两点原因：

1. 最小化经验风险很容易导致过拟合，因为最小化会让模型过于关注训练集，偏差过大；


2. 很多优化算法是基于梯度下降的，但是很多有用的损失函数，如 0-1 损失，没有有效的导数，无法直接用于优化算法。

下面我们会介绍在机器学习中有效的优化方法。

***
### 代理损失函数和提前终止

上面提到的两点不足可以通过以下两个方法来解决：

1. **使用代理损失函数**：

    面对类似 0-1 损失这种不可解问题，我们通常优化**代理损失函数**。它作为原目标的代理，会比原损失函数学到更多。

    比如使用对数似然替代原函数时，在训练集上 0-1 损失达到 0 之后，测试集上的 0-1 还能下降很长一段时间，这是因为即使训练集上的 0-1 损失为 0，还能拉开不同类别的距离以增加分类器的健壮性，得到一个更强壮的分类器。
    
    所以，相对于简单地最小化训练集上的平均 0-1 损失，它能够从训练数据中抽取更多信息。
    
    
2. **提前终止**：

    区别于传统优化算法，训练算法一般不会停止在局部极小点，而是基于提前终止的收敛条件满足时停止优化代理损失函数。

    一般，提前终止使用真实潜在损失函数，比如验证集上的 0-1 损失，并设计在过拟合发生前终止，参见上一章。

***
### 批量算法和小批量算法

机器学习算法 和一般优化算法不同的地方在于，**机器学习算法的目标函数通常可以分解为训练样本上的求和。**所以实践中在每一次更新优化算法时通常仅使用整个代价函数中的部分项来估计代价函数的期望值。

这样做的理由是什么呢？我们结合最大似然估计问题来解释这件事情。

最大似然问题在对数空间中分解成各个样本的总和：

$$\mathbf{\theta}_{ML} = \arg\max_{\mathbf{\theta}}\sum_{i=1}^m \log p_{model}(\mathbf{x}_i,y_i;\mathbf{\theta})$$

由中心极限定理可知，上式等价于：

$$J(\mathbf{\theta}) = E_{\;(\mathbf{x},y)\sim \hat{p}_{data}}\log p_{model}(\mathbf{x}_i,y_i;\mathbf{\theta})$$

优化算法中最常用的属性是基于训练集上的样本梯度的期望：

$$\nabla_{\mathbf{\theta}}J(\mathbf{\theta}) = E_{\;(\mathbf{x},y)\sim \hat{p}_{data}}\nabla_{\mathbf{\theta}}\log p_{model}(\mathbf{x}_i,y_i;\mathbf{\theta})$$

这个期望的计算代价非常大，因为要在整个训练集的每个样本上评估模型。实践中，我们从数据集中**随机**采样少量样本，然后计算样本上的平均值。

这样做有以下两方面理由：

* **从计算量而不是更新次数角度而言，收敛更快**：

    抽取 n 个样本均值的标准差是 $\frac{\sigma}{\sqrt{n}}$(参考[数学公式](https://github.com/BofengChen/ml_notes/blob/master/CH0_%E6%95%B0%E5%AD%A6%E5%85%AC%E5%BC%8F.ipynb)的“均值的标准差”章节),其中 $\sigma$ 是样本值真实的标准差。分母表明使用更多样本来估计梯度方法的汇报是低于线性的，比如基于 100 个样本和 10000 个样本的梯度计算，后者的计算量是前者的100倍，但是只降低了10倍的均值标准差。

    如果可以快速计算出梯度估计值，而不是缓慢计算正确值，大多数优化算法会收敛更快(在总的计算量的意义上，而不是更新次数)。两种计算思路的差异类似于 做加法 和 做乘法 的区别。
    

* **训练集的冗余导致重复**：

    最坏的情况下，训练集中的 m 个样本都是相同的拷贝，基于采样的梯度估计可以使用单个样本计算出正确的梯度，比原来的做法少花了 m 倍时间。
    
    实践中，不太会遇到这种最坏情况，但可能会发现大量样本都对梯度做出了非常相似的贡献。

使用整个训练集的优化算法被称为 **批量(batch)梯度算法**，每次只使用单个样本的优化算法被称为 **随机(在线)算法**。“在线”通常是指在连续产生样本的数据流中抽取样本，而不是从一个固定大小的训练集中遍历多次采样。

大多数用于深度学习的算法介于以上两者之间，使用介于一个样本和全部样本之间数量的训练样本。我们称这种为**小批量随机算法**，或者简单称为**随机算法**。

***
**小批量的尺寸**通常由以下几个因素决定：

* 大批量会计算出更精确的梯度估计，但是回报小于线性；


* 极小批量尺寸难以充分利用多核架构，促使我们给出一个绝对最小批量的尺寸，低于这个尺寸不会减少计算时间；


* 如果批量处理的所有样本可以并行处理，那么内存消耗与批量尺寸成正比，所有硬件设施会成为批量尺寸的限制因素；


* 在某些硬件上，尺寸为具有某种特定模式的数组时，运行时间会更少。尤其在使用 GPU 时，通常以 2 的幂数 作为批量的尺寸，取值范围介于 32-256之间，幂数取 16 有时会在尝试大型模型时使用；


* 可能是由于小批量在学习过程中加入了噪声，会有一些正则化效果。但太小会导致学习率也必须很小来保持稳定性，偏低的学习率和更多的步骤会导致总的运行时间非常大，所以批量尺寸不能过小。

**小批量的随机抽取**

小批量是随机抽取这一点也很重要，这样可以保证两个连续的梯度估计是互相独立的，所以两个连续的小批量样本应该是彼此独立的。实践中通常将样本顺序打乱一次，然后按照这个顺序存储起来就足够了。

**在线学习**

在使用一个非常大的训练集时，过拟合不再是问题，而欠拟合和计算效率变成了主要的顾虑。所以随着数据集的规模迅速增长，超越了计算能力的增速，机器学习训练中每个样本只使用一次，甚至只使用部分训练集的情况越来越常见。

## 优化中的挑战

训练神经网络时，肯定会遇到非凸情况，即使是凸优化，也并非没有任何问题。下面我们会总结几个训练深度模型时遇到的主要挑战。

### 病态

优化凸函数，会遇到 Hessian 矩阵 $\mathbf{H}$ 是病态的情形，也就是条件数很大的情形。病态问题一般存在于神经网络训练过程中，病态体现在随机梯度会“卡”在某些情形下，此时即使很小的更新步长也会增加代价函数。

除了神经网络训练，病态问题还存在于其他问题和情形中，有些适用于其他情况下解决病态问题的技术并不适合神经网络的训练过程。比如，**牛顿法**在解决带有病态条件的 Hessian 矩阵的凸优化问题时，是一个非常好的方法，但是要运用到神经网络中需要很大的改动，之后的小节会介绍。

### 局部极小值

凸优化问题的一个突出特点是可以简化为寻找一个局部极小点的问题，局部极小点就是全局最小点。

对于非凸函数，如神经网络，可能会存在多个局部极小值。几乎所有的深度模型都会有非常多的局部极小值，但这个并不仅仅是因为非凸所引起的，还有可能是因为**权重空间对称性**所带来的。权重空间对称性是指我们可以通过交换神经单元 i 和单元 j 的传入权重向量、传出权重向量而得到等价的模型。因为这种对称性会出现多个等效的模型，存在多个局部极小值。

对于实际中的网络，是否存在大量代价很高的局部极小值，优化算法是否会碰到这些局部极小值，都是尚未解决的公开问题。

### 高原、鞍点和其他平坦区域

对于很多高维非凸函数，局部极小值(以及极大值)事实上都远少于另一类梯度为零的点：**鞍点**。在鞍点处，Hessian 矩阵同时具有正负特征值。

***
很多随机函数有如下性质：

* **低维空间中，局部极小值很普遍，高维空间中，极小值很罕见，鞍点很常见。**

    从概率上讲，局部极小值对应的 Hessian 矩阵只有正的特征值，鞍点处的Hessian 矩阵同时有正负特征值。把Hessian 矩阵的正负特征值由抛硬币决定，一维情况下很容易抛硬币得到正面朝上一次而获取到局部极小值，但是在 n-维空间，要抛掷 n 次硬币都正面朝上的难度是指数级的。


* **当我们达到代价函数取值较低的区间时，Hessian 矩阵的特征值为正的可能性更大。同样以抛硬币为类比，意味着当我们处于低代价的临界点时，抛掷硬币正面 n 次朝上的概率更大。**

    这也意味着，局部极小值具有低代价的可能性比高代价要大得多，具有高代价的临界点更有可能是鞍点。具有极高代价的临界点很有可能就是局部极大值。

***

以上出现在随机函数中的现象，是否也会在神经网络中出现呢？对于不具有非线性的浅层自编码器，只有全局极小值和鞍点，没有其他极小值点。对于真实的神经网络，有实验表明也存在包含很多高代价鞍点的损失函数。

**鞍点激增对训练算法的影响**：

* 对于只使用梯度信息的一阶优化算法而言，影响还不是很清楚，鞍点附近的梯度通常会非常小。实验中梯度下降似乎可以再许多情况下逃离鞍点(感性的结论)；


* 对于涉及到二阶 Hessian 矩阵的牛顿法，鞍点会是一个问题。梯度下降旨在朝“下坡”移动，而非寻求明确的临界点，而牛顿法的目标是寻求梯度为零的点，如果不进行适当的修改，牛顿法就会跳进一个鞍点。

    高维空间鞍点的激增或许解释了在神经网络训练中为何二阶方法无法成功取代梯度下降。Dauphin 介绍的二阶优化的 **无鞍牛顿法**表名比传统算法有显著改进。

***
**极大值**

除了极小值和鞍点，也存在其他梯度为零的点，比如极大值。除了未经修改的牛顿法，很多算法不会被吸引到极大值。和极小值一样，高维情形下许多种类的随机函数的极大值指数级稀少。

**平坦区域**

在宽且平坦的区间，梯度和 Hessian 矩阵都是零。凸问题中，一个宽且平坦的区域肯定包含全局极小值，但是对于一般的优化问题而言，这样的区域可能对应着目标函数中一个代价较高的值。

### 悬崖和梯度爆炸

多层神经网络通常存在像悬崖一样的斜率较大的区域，这是由于几个较大的权重相乘导致的。**遇到斜率极大的悬崖结构时，梯度更新会很大程度改变参数值，通常会完全跳过这类悬崖结构。**

<img src='figure/xuanya0.png' width='400' height='300'>

无论我们从上还是从下接近悬崖，情况都很糟糕，但幸运的是，我们可以使用之后章节介绍的启发式**梯度截断**来避免其严重后果：

    基本想法来源于梯度并没有指名最佳步长，只说明了无限小区域内的最佳方向。当传统的梯度下降算法提议更新很大一步时，启发式梯度截断会来干涉减小步长，从而使其不太可能走出梯度近似为最陡下降方向的悬崖区域。
    
悬崖结构在循环神经网络的代价函数中很常见，因为这类模型会涉及多个因子相乘，其中每个因子都对应一个时间步，因此长期时间序列会产生大量相乘。

### 长期依赖

当计算图很深时，神经网络优化算法会面临另一个难题是**长期依赖问题——由于变深的结构使得模型丧失了学习到先前信息的能力，让优化变得及其困难。**

假设某个计算图包含一条反复与矩阵 $\mathbf{W}$ 相乘的路径，那么 t 步后相当于乘以 $\mathbf{W}^t$。 假设 $\mathbf{W}$ 有特征分解 $\mathbf{W} = \mathbf{V}diag(\mathbf{\lambda})\mathbf{V}^{-1}$。在这种简单情况下，容易看出：

$$\mathbf{W}^t = \mathbf{V}diag(\mathbf{\lambda})^t\mathbf{V}^{-1}$$

当特征值 $\lambda_i$ 不在 1 的附近时，若在量级上大于 1 则会爆炸；若小于 1 时会消失。

**梯度消失**与**梯度爆炸**是指该计算图上的梯度也会因为 $diag(\mathbf{\lambda})^t$ 大幅度变化：

* 梯度消失使得我们很难知道参数朝哪个方向移动能够改进代价函数；


* 梯度爆炸会使得学习不稳定，之前描述的促使我们使用梯度截断的悬崖结构就是梯度爆炸现象的一个例子。

### 局部和全局结构之间的弱对应

上面我们介绍的各种优化过程中遇到的挑战都是关于损失函数在单点的性质——若 $J(\mathbf{\theta})$ 在点 $\mathbf{\theta}$ 有病态条件，或者 $\mathbf{\theta}$ 在悬崖中， 或者 $\mathbf{\theta}$ 是一个下降方向不明显的鞍点，都会很难更新当前步。

但是如果该方向在局部改进很大，但并没有指向代价低得多的遥远区域，那么我们可能在单点处克服以上所有困难，但仍然表现不佳。

下图就是一个例子，即使没有局部极小值和鞍点这类单点处会出现的优化上的挑战，但是因为全局结构上的原因，还是不能从局部优化中找到一个良好的代价函数值。

许多现有研究方法在求解有困难全局结构的问题时，**旨在寻求良好的初始点，而不是开发非局部范围更新的算法。**我们的建议也是，在传统的优化算法中研究怎么选择更加的初始化点，以此来实现目标更切实可行。

<img src='figure/quanju0.png' width='400' height='300'>

## 参数初始化策略

### 重要性

深度学习模型的训练算法通常是迭代的，因此要求指定一些开始迭代的初始点。而且训练深度模型是一个足够困难的问题，所以大多数算法都很大程度收到**初始化选择**的影响，这也就是**参数初始化策略的重要性所在**。

初始化策略能够决定算法是否收敛并且决定学习收敛的速度，以及收敛到一个代价高还是低的点。并且，对应代价相近的不同参数所对应的泛化误差差别很大，即初始参数会影响泛化。

***

### 认知

现代的初始化策略是启发式的，没有很好的理论基础。大多数初始化策略基于在神经网络初始化时实现一些很好的性质，然而问题在于：

* 我们无法确保哪些好的性质会在学习开始后的哪些情况下得以保持；

* 另外潜在的风险是，有些初始点从优化问题本身的角度看是有益的，但是从泛化的观点是不利的。

总之，我们几乎没有任何关于初始化策略的理论上的指导。唯一可以确定的是**初始化参数需要在不同单元间“破坏对称性”。**

如果两个单元具有相同的初始参数，应用确定性损失和模型的确定性学习算法，会导致一直以相同的方式更新这两个单元。这样不利于找到所有的输入模式或者梯度模式(还没完全理解)。每个单元需要计算不同函数的目标促使了参数的随机初始化。

***

### 策略和思想

#### 权重参数

我们总是通过**高斯或者均匀分布随机抽取值**来作为初始化模型的权重。具体是选择高斯还是均匀分布似乎差别不大，但也没有理论上的解释。

下面我们要特别来说明下**初始分布的值的大小对优化过程或者网络泛化能力大小都有很大的影响。**

***
**影响优化过程中权重的理想值的因素**

**初始权重值更大的优点：**

* 初始权重值更大对单元的对称性有更强的破坏性，有助于避免冗余的单元；


* 大权重也有助于避免在每层线性成分的前向/后向传播中丢失信号——矩阵中更大的值在矩阵乘法中有更大的输出。

**初始权重值更大的缺点：**

* 如果初始权重值过于大，会在前向/后向传播中产生爆炸的值(接近无穷的值)；


* 很大的权重会导致**混沌**(对于输入中很小的扰动非常敏感，导致确定性前向传播过程表现地很随机)。这里面出现的梯度爆炸问题可以通过梯度截断来缓解；


* 很大的权重也会容易产生使激活函数饱和的值，导致饱和单元的梯度完全丢失。


***
**正则化和优化对于初始化网络权重的相反视角**

上面的优化观点建议权重应该足够大来成功传播信息，但是正则化希望权重小一点。SGD这类考虑了正则化的优化算法，倾向于最终参数应该停止在更靠近初始参数的地方(比如提前终止的梯度下降以及 L2/L1 正则化的权重衰减)。

***

**启发式方法选择权重初始大小**

一种初始化 m 个输入和 n 个输出的全连接层权重的启发式方法是从分布 $U(-\frac{1}{\sqrt{m}},\frac{1}{\sqrt{m}})$ 中采样权重， Glorot and Bengio 建议使用如下的**标准初始化**：

$$W_{i,j}\sim \left(-\sqrt{\frac{6}{m+n}},\sqrt{\frac{6}{m+n}}\right)$$

Saxe 推荐初始化为随机正交矩阵，仔细挑选负责每一层非线性缩放或者 **增益因子** g(??)。

***

如果计算资源允许，将每层权重的初始数值范围视为**超参数**通常是个好主意，使用后面章节介绍的超参数搜索算法，如随机搜索，挑选这些数值范围，是否选择使用密集或者稀疏初始化也可以作为一个超参数。

如果计算资源不够，我们可以手动搜索最优初始范围，一个好的经验法则是**观察单个小批量数据上的激活函数或梯度的幅度或标准差**：

    如果权重太小，那么当激活值在小批量上前向传播时，激活值的幅度会缩小。通过重复识别神经网络中具有小得不可接受的激活值的第一层，并提高其权重，最终有可能得到一个初始激活全部合理的网络。
    
    如果学习在激活上仍然很慢，可以观测梯度的幅度或者标准差会有所帮助。这个过程的计算量低于基于验证集误差的超参数优化，因为它是基于初始模型在单批数据上的行为反馈，而不是验证集上训练模型的反馈，总的来说是因为数据规模的量级小。

#### 偏置及其他参数

一般情况下，我们可以为每个单元的**偏置**设置启发式挑选的常数。

设置偏置的方法必须和设置权重的方法协调，**设置偏重为零通常在大多数权重初始化方案里都是可行的，需要设置为非零值的情况等我理解了再介绍。**比如LSTM中遗忘门的偏置设置为1。

其他参数通常和偏差一样为启发式选择的常数。

## 基本优化算法

参考[机器学习最优化理论](https://github.com/BofengChen/ml_notes/blob/master/CH2_%E6%9C%80%E4%BC%98%E5%8C%96%E7%90%86%E8%AE%BA.ipynb)部分的“随机梯度下降”、“动量”以及“Nesterov动量”章节。

## 自适应学习率算法

学习率是难以设置的超参数之一，因为它对模型的性能有显著的影响。正如我们之前所说，损失通常高度敏感于参数空间中的某些方向，而不敏感于其他（比如不同的特征根对应的特征向量所在方向）。动量算法可以一定程度上缓解这一问题，但这样做的代价是引入另一个超参数。有没有其他办法？

如果我们相信方向敏感度在某种程度是轴对齐的，那么为每个参数设置不同的学习率，在整个学习过程中自动适应这些学习率是合理的。下面会介绍几种自适应学习算法。

### AdaGrad

AdaGrad 算法，如下图所示，独立地适应所有模型参数的学习率，参数的分量缩放与在历史迭代中梯度的相应分量累计平方和的平方根成反比：

$$\begin{split}
& \mathbf{r}^t\leftarrow \mathbf{r}^{t-1} + \mathbf{g}^t\odot \mathbf{g}^t\\
&\Delta\mathbf{\theta}^t\leftarrow -\frac{\epsilon}{\delta+\sqrt{\mathbf{r}^t}}\odot \mathbf{g}^t\\
&\mathbf{\theta}^t\leftarrow \mathbf{\theta}^{t-1} +\Delta\mathbf{\theta}^t\\
\end{split}$$

其中

$$\Delta\theta_i^t = -\frac{\epsilon}{\delta+\sqrt{r_i^t}}g_i^t$$

t 表示迭代轮数，i 表示参数的相应分量，$\odot$ 表示 Hadamard 乘积。

由 
$$r_i^t = \sum_{k=1}^{t-1}(g_i^k)^2$$ 
可知新的学习率

$$\tilde{\epsilon}^t = \frac{\epsilon}{\delta+\sqrt{r_i^t}}$$

关于梯度 $\mathbf{g}^t$ 和 新的学习率 $\tilde{\epsilon}^t$ 有如下结论：

* 梯度对应的最大分量随着 t 的增加如果一直很大，随着 t 的增加 $\tilde{\epsilon}^t$ 的下降速度很快,进而参数 $\mathbf{\theta}^t$ 相应分量的减小会变慢；


* 梯度对应的最小分量随着 t 的增加如果一直很小，随着 t 的增加 $\tilde{\epsilon}^t$ 的下降速度很慢，进而参数 $\mathbf{\theta}^t$ 相应分量的减小会变快。

综上可知，净效果是在参数空间上更为平缓的倾斜方向会取得很大的前进。

在凸优化背景中(可以简单理解为 梯度的方向在单侧是单调的)， AdaGrad 算法有一些令人满意的理论性质。然而经验发现，对于训练深度神经网络模型而言，从**训练开始时积累梯度平方会导致有效学习率过早和过量的减小**。AdaGrad 在某些向深度学习模型上效果不错，但不是全部。

下面是算法流程图：

<img src='figure/AdaGrad0.png' ailgn='left'>

### RMSProp

RMSProp 算法(Hinton)修改 AdaGrad 使得其在非凸设定下效果也好。核心的不同在于将 AdaGrad 中的梯度积累改为指数加权的移动平均。这种改善方式其背后的思想是：

    当问题是非凸时，如果还按照 AdaGrad 设定为梯度积累，会过度依赖于历史梯度的累积，因为问题是非凸的，学习轨迹可能穿过很多不同的结构，这种情况下历史梯度的累积并不一定起到积极的作用，反而有可能在达到凸结构之前就变得过小；
    
    RMSProp 使用指数衰减平均以丢弃遥远过去的历史，使其能够在找到凸碗状结构后快速收敛。
    
经验上， RMSProp 已经被证明是一种有效且实用的深度神经网络优化算法。

下面分别是 RMSProp 的标准形式 以及 结合了 Nesterov 动量的 RMSProp 形式。相比于 AdaGrad,使用移动平均引入了一个新的超参数 $\rho$ 来控制移动平均的长度范围。

<img src='figure/RMSProp0.png'>

<img src='figure/NesterovAndRMSProp0.png'>

疑问：在 `Nesterov 动量的 RMSProp 形式` 中不需要考虑加小常数来保持数值稳定么？

### Adam

Adam 是另一种学习率自适应的优化算法，如下图所示。

<img src='figure/Adam0.png'>

"Adam" 这个词派生于短语“adaptive moments”。早期的算法背景下，被看做结合了 RMSProp 和 动量的变种：

* 在 Adam 中，动量直接并入了梯度一阶矩（指数加权）的估计；


* Adam 包含偏置修正，修正从原点初始的二阶矩估计；RMSProp 也采用了(非中心的)二阶矩估计，但缺失了修正因子，因此 RMSProp 二阶矩估计在训练初期会有很高的偏置。

Adam 通常被认为对超参数的选择有很好的鲁棒性，尽管学习率有时需要从建议的默认范围进行修改。

### 选择正确的优化算法

没有理论上的共识与指导，最流行并且使用很高的优化算法包括 SGD、具有动量的 SGD、RMSProp、具有动量的 RMSProp、AdaDelta 和 Adam。

## 二阶近似方法

在本节中，我们常会讨论训练深度神经网络中的二阶方法。为了表述的简单，我们只考察目标函数为经验风险：

$$J(\mathbf{\theta}) = E_{\mathbf{x},y\sim \hat{p}_{data}\;(\mathbf{x},y)}[L(f(\mathbf{x};\mathbf{\theta}),y)]
=\frac{1}{m}\sum_{i=1}^m L(f(\mathbf{x}_i;\mathbf{\theta}),y_i)$$

具体的介绍参考[机器学习最优化理论](https://github.com/BofengChen/ml_notes/blob/master/CH2_%E6%9C%80%E4%BC%98%E5%8C%96%E7%90%86%E8%AE%BA.ipynb)部分的“牛顿法”、“共轭梯度”以及“BFGS”章节。

## 优化策略和元算法

许多优化技术并非真正的算法，而是一般化的模板，可以根据特定的问题来产生算法，或是并入到很多不同的算法中。

### 批标准化

### 坐标下降法

一些情况下，将一个优化问题分解成几个独立的部分，可以更快解决原问题。

如果我们相对于某个单一变量 $x_i$ 最小化 $f(\mathbf{x})$ ，然后相对于另一个变量 $x_j$ 等反复循环所有的变量，我们会保证到达(局部)极小值。这种做法被称为**坐标下降(coordinate descent)**。

更一般地，**块坐标下降(block coordinate descent)** 是指对于某个子集的变量同时最小化。术语“坐标下降”通常指块坐标下降，也指严格的单个坐标下降。

当一个变量的值很大程度地影响另一个变量的最优值时，坐标下降不是一个很好的方法。

### Polyak 平均

Polyak 平均会平均优化算法在参数空间访问轨迹中的几个点。如果 t 次迭代梯度下降访问了点 $\mathbf{\theta}_1$、$\cdots$、$\mathbf{\theta}_t$，那么 Polyak 平均算法的输出是 $\hat{\mathbf{\theta}}_t=\frac{1}{t}\sum_{i}\mathbf{\theta}_i$。

如梯度下降应用于凸问题时，这种方法具有较强的收敛保证。应用于神经网络时，其验证更多是启发式的，但实践中表现良好。基本想法是：

    优化算法可能会来回穿过山谷好几次而没有经过山谷底部附近的点，两边所有位置的均值应比较接近谷底。

在非凸问题中，优化轨迹的路径非常复杂，并且经过很多不同的区域。包括参数空间中遥远过去的点，可能与当前点在代价函数上相隔很大的障碍。

基于这种现象，当 Polyak 应用于非凸问题时通常会用指数衰减来计算平均值，可以抛弃遥远历史上的点：

$$\hat{\mathbf{\theta}}_t = \alpha \hat{\mathbf{\theta}}_{t-1} +(1-\alpha)\hat{\mathbf{\theta}}_t$$

### 监督预训练

如果模型太复杂难以优化或者任务非常困难，直接训练模型来解决特定任务的难度太大。这种情形下，我们可以通过：

* **训练一个简单模型来求解问题，然后使得模型更复杂更有效；**


* **训练模型来求解一个简单的问题，然后转移到最后的问题。**

总的来说，可以采取“简单模型+复杂问题”或者“复杂模型+简单问题”来进行预训练。

这些在直接训练目标模型来求解目标问题之前，训练简单模型来求解简化问题的方法统称为**预训练**。

***

**贪心监督预训练**

**贪心算法**可以将问题分解成很多部分，然后独立每个部分求解最优值，之后可以紧接一个**精调**阶段，联合优化算法搜索全问题的最优解。**使用贪心解初始化联合优化算法，可以极大加速算法，并提高找到的解的质量。**

预训练算法，特别是贪心预训练，在深度学习中是普遍存在的。我们下面会具体描述这些将监督学习问题分解成其他简化的监督学习问题的预训练算法。这种方法被称为**贪心监督预训练**。

### 设计有助于优化的模型

实践中，选择一族易优化的模型比使用一个强大的优化算法更重要。过去30年神经网络的大多数进步主要来自改变模型族，而非优化过程。早年用于训练神经网络的带动量的随机梯度下降，仍然是现代神经网络应用中的前沿算法。

现代神经网络的**设计选择**体现在：

* 层之间的线性变换。

    线性变换的 jacobian 具有相对合理的奇异值，则梯度可以流经很多层；
    
    此外，线性函数在一个方向上一致增加，即使模型的输出远离正确值，也可以简单清晰地计算梯度，使其输出方向朝降低损失函数的方向移动。


* 几乎处处可导的激活函数；


* 大部分定义域都有确切定义的梯度。


总的来说，现代神经网络的设计方案旨在使其**局部梯度信息**合理地朝着正确的解移动。

### 延拓法与课程学习